In [1]:
! pip install networkx matplotlib

import networkx as nx
from matplotlib import animation, pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import time


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\anmol\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
#for simplicity we ll consider heuristic distances given and this function returns heuristic distance for all nodes
def heuristic(n):
    H_dist = {
        'S': 13,
        'A': 11,
        'B': 6,
        'C': 99,
        'D': 1,
        'E': 7,
        'G': 0,
    }
    return H_dist[n]

#Describe your graph here
Graph_nodes = {
    'S': [('A', 1), ('C', 5)],
    'A': [('S', 1), ('B', 2), ('E', 3)],
    'B': [('A', 2), ('C', 1), ('G', 9)],
    'C': [('S', 5), ('B', 1)],
    'D': [('E', 6), ('G', 1)],
    'E': [('A', 3), ('D', 6)],
    'G': [('B', 9), ('D', 1)]
}


In [3]:
def RBFS_Anim(start_node, stop_node):
    
    open_set = set([start_node])
    closed_set = set()
    g = {}
    h = {}
    parents = {}
    
    g[start_node] = 0
    h[start_node] = heuristic(start_node)
    parents[start_node] = start_node
    
    # Initialize animation data
    animation_data = {
        'open_sets': [],
        'closed_sets': [],
        'current_nodes': [],
        'parents_maps': [],
        'path_costs': [],
        'heuristic_costs': [],
        'f_limits_alter_costs': []
    }

    
    def rbfs_recursive(current_node, f_limit_2nd, open_set, closed_set, depth=0):
        
        # Store current state for animation
        animation_data['open_sets'].append(open_set.copy())
        animation_data['closed_sets'].append(closed_set.copy())
        animation_data['current_nodes'].append(current_node)
        animation_data['parents_maps'].append(parents.copy())
        animation_data['path_costs'].append(g.copy())
        animation_data['heuristic_costs'].append(h.copy())
        animation_data['f_limits_alter_costs'].append(f_limit_2nd)
        
        print(f"RBFS Depth {depth}: Node {current_node}, f-limit (2nd alter): {f_limit_2nd}")
        
        if current_node == stop_node:
            # Reconstruct path
            path = []
            temp_node = current_node
            while parents[temp_node] != temp_node:
                path.append(temp_node)
                temp_node = parents[temp_node]
            path.append(temp_node)
            path.reverse()
            print(f'Path found against the defined closed frontier list : {closed_set}, for child:parent defined {parents}.')
            return path, g[current_node]
        
        # Generate successors
        successors = []
        if current_node in Graph_nodes:
            for (next_node, weight) in Graph_nodes[current_node]:
                if next_node not in closed_set:
                    g_new = g[current_node] + weight
                    f_new = g_new + heuristic(next_node)
                    successors.append((g[current_node] + heuristic(next_node) + weight, g[current_node] + weight, next_node))
                    
                    if next_node not in open_set:
                        open_set.add(next_node)
                        parents[next_node] = current_node
                        g[next_node] = g[current_node] + weight
                        h[next_node] = heuristic(next_node)
                    else:
                        if g_new < g[next_node]:
                            g[next_node] = g[current_node] + weight
                            h[next_node] = heuristic(next_node)
                            parents[next_node] = current_node
        
        if not successors:
            return None, float('inf')
        
        # Sort successors by f-value
        successors.sort(key=lambda x: x[0])
        
        while successors:
            best_f, best_g, best_node = successors[0]
            
            # Store state before recursive call
            animation_data['open_sets'].append(open_set.copy())
            animation_data['closed_sets'].append(closed_set.copy())
            animation_data['current_nodes'].append(current_node)
            animation_data['parents_maps'].append(parents.copy())
            animation_data['path_costs'].append(g.copy())
            animation_data['heuristic_costs'].append(h.copy())
            animation_data['f_limits_alter_costs'].append(f_limit_2nd)
            
            if best_f > f_limit_2nd:
                return None, best_f
            
            # Get alternative f-value for 2nd best alternative for cost comparison
            alternative_f = successors[1][0] if len(successors) > 1 else float('inf')
            
            # Update sets for recursion
            open_set.remove(best_node)
            closed_set.add(best_node)
            
            # Recursive call - (may by optimized using Bottom-Up Dynamic Programming logics via a memoization array)
            result, result_f = rbfs_recursive(best_node, min(f_limit_2nd, alternative_f), open_set, closed_set, depth + 1)
            
            # Store state after recursive call
            animation_data['open_sets'].append(open_set.copy())
            animation_data['closed_sets'].append(closed_set.copy())
            animation_data['current_nodes'].append(current_node)
            animation_data['parents_maps'].append(parents.copy())
            animation_data['path_costs'].append(g.copy())
            animation_data['heuristic_costs'].append(h.copy())
            animation_data['f_limits_alter_costs'].append(f_limit_2nd)
            
            if result is not None:
                return result, result_f
            
            # Backtrack
            closed_set.remove(best_node)
            open_set.add(best_node)
            
            # Update f-value and re-sort
            successors[0] = (result_f, g[best_node], best_node)
            successors.sort(key=lambda x: x[0])
            
            # Update g value
            g[best_node] = result_f - heuristic(best_node)
        
        return None, float('inf')
    
    # Initialize sets for RBFS
    open_set = set([start_node])
    closed_set = set()
    
    # Start RBFS
    path, cost = rbfs_recursive(start_node, float('inf'), open_set, closed_set)
    
    if path:
        print('Path found via RBFS: {}'.format(path))
        print('Total cost: {}'.format(cost))
        return path, animation_data
    else:
        print('Path does not exist!')
        return None, animation_data

RBFS_Anim('S','G')

RBFS Depth 0: Node S, f-limit (2nd alter): inf
RBFS Depth 1: Node A, f-limit (2nd alter): 104
RBFS Depth 2: Node B, f-limit (2nd alter): 11
RBFS Depth 2: Node E, f-limit (2nd alter): 12
RBFS Depth 3: Node D, f-limit (2nd alter): 12
RBFS Depth 4: Node G, f-limit (2nd alter): 12
Path found against the defined closed frontier list : {'D', 'G', 'A', 'E'}, for child:parent defined {'S': 'S', 'A': 'S', 'C': 'B', 'B': 'A', 'E': 'A', 'G': 'D', 'D': 'E'}.
Path found via RBFS: ['S', 'A', 'E', 'D', 'G']
Total cost: 11


(['S', 'A', 'E', 'D', 'G'],
 {'open_sets': [{'S'},
   {'A', 'C', 'S'},
   {'C', 'S'},
   {'B', 'C', 'E', 'S'},
   {'C', 'E', 'S'},
   {'C', 'E', 'G', 'S'},
   {'C', 'E', 'G', 'S'},
   {'B', 'C', 'E', 'G', 'S'},
   {'B', 'C', 'G', 'S'},
   {'B', 'C', 'D', 'G', 'S'},
   {'B', 'C', 'G', 'S'},
   {'B', 'C', 'G', 'S'},
   {'B', 'C', 'S'},
   {'B', 'C', 'S'},
   {'B', 'C', 'S'},
   {'B', 'C', 'S'},
   {'B', 'C', 'S'}],
  'closed_sets': [set(),
   set(),
   {'A'},
   {'A'},
   {'A', 'B'},
   {'A', 'B'},
   {'A', 'B'},
   {'A'},
   {'A', 'E'},
   {'A', 'E'},
   {'A', 'D', 'E'},
   {'A', 'D', 'E'},
   {'A', 'D', 'E', 'G'},
   {'A', 'D', 'E', 'G'},
   {'A', 'D', 'E', 'G'},
   {'A', 'D', 'E', 'G'},
   {'A', 'D', 'E', 'G'}],
  'current_nodes': ['S',
   'S',
   'A',
   'A',
   'B',
   'B',
   'A',
   'A',
   'E',
   'E',
   'D',
   'D',
   'G',
   'D',
   'E',
   'A',
   'S'],
  'parents_maps': [{'S': 'S'},
   {'S': 'S', 'A': 'S', 'C': 'S'},
   {'S': 'S', 'A': 'S', 'C': 'S'},
   {'S': 'S', 'A': 'S'

In [1]:
def animate_RBFS_search(animation_data, final_path=None):
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))
    plt.subplots_adjust(wspace=0.3)
    
    # Create the complete graph for reference
    complete_G = nx.Graph()
    for node, neighbors in Graph_nodes.items():
        for neighbor, weight in neighbors:
            complete_G.add_edge(node, neighbor, weight=weight)
    
    # Precompute positions for consistent layout
    all_nodes = set()
    for parents_map in animation_data['parents_maps']:
        all_nodes.update(parents_map.keys())

    # Use spring layout with fixed seed for consistent positioning
    pos = nx.spring_layout(complete_G, seed=42)
    
    def update(frame):
        ax1.clear()
        ax2.clear()
        
        if frame >= len(animation_data['open_sets']):
            return
        
        # Current state
        open_set = animation_data['open_sets'][frame]
        closed_set = animation_data['closed_sets'][frame]
        current_node = animation_data['current_nodes'][frame]
        parents = animation_data['parents_maps'][frame]
        path_costs = animation_data['path_costs'][frame]
        heuristic_costs = animation_data['heuristic_costs'][frame]
        f_limit_2nd = animation_data['f_limits_alter_costs'][frame]
        
        # Plot 1: Current search tree based on parents
        G = nx.DiGraph()
        all_nodes = set(parents.keys())
        for current_node in all_nodes:
            G.add_node(current_node)
        
        for child, parent in parents.items():
            if child != parent:
                weight = 0
                if parent in Graph_nodes:
                    for neighbor, w in Graph_nodes[parent]:
                        if neighbor == child:
                            weight = w
                            break
                G.add_edge(parent, child, weight=weight)
        
        # Node colors
        node_colors = []
        node_labels = {}
        for node in G.nodes():
            if node == current_node:
                node_colors.append('yellow')  
                node_labels[node] = f"{node}\nCURRENT"
            elif node in open_set:
                node_colors.append('lightgreen')  # Open set
                node_labels[node] = f"{node}\nOPEN"
            elif node in closed_set:
                node_colors.append('lightcoral')  # Closed set
                node_labels[node] = f"{node}\nCLOSED"
            else:
                node_colors.append('lightblue')  # Unexplored
                node_labels[node] = node
        
        # Draw graph
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=800, ax=ax1)
        nx.draw_networkx_edges(G, pos, edge_color='gray', arrows=True, arrowsize=20, ax=ax1)
        nx.draw_networkx_labels(G, pos, ax=ax1)
        
        # Use custom labels for better readability
        nx.draw_networkx_labels(G, pos, labels=node_labels, ax=ax1)
        
        # Edge labels (weights)
        edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax1)
        
        ax1.set_title(f'RBFS Step {frame+1}\nCurrent: {current_node}, F-limit(2nd alter): {f_limit_2nd}')
        ax1.set_xlim(-1.5, 1.5)
        ax1.set_ylim(-1.5, 1.5)
        
        # Plot 2: Complete graph with status and costs
        complete_colors = []
        for node in complete_G.nodes():
            if node == current_node:
                complete_colors.append('yellow')
            elif node in open_set:
                complete_colors.append('lightgreen')
            elif node in closed_set:
                complete_colors.append('lightcoral')
            else:
                complete_colors.append('lightblue')
        
        nx.draw_networkx_nodes(complete_G, pos, node_color=complete_colors, node_size=800, ax=ax2)
        nx.draw_networkx_edges(complete_G, pos, ax=ax2)
        nx.draw_networkx_labels(complete_G, pos, ax=ax2)
        
        # Add detailed node labels with costs
        node_labels = {}
        for node in complete_G.nodes():
            g_val = path_costs.get(node, 'Infinity')
            h_val = heuristic(node)
            f_val = g_val + h_val if isinstance(g_val, (int, float)) else 'Infinity'
            node_labels[node] = f'{node}\ng:{g_val}\nh:{h_val}\nf:{f_val}'
        
        label_pos = {k: (v[0], v[1] + 0.1) for k, v in pos.items()}
        nx.draw_networkx_labels(complete_G, label_pos, labels=node_labels, ax=ax2)
        
        # Highlight final path
        if final_path and frame == len(animation_data['open_sets']) - 1:
            path_edges = [(final_path[i], final_path[i+1]) for i in range(len(final_path)-1)]
            nx.draw_networkx_edges(complete_G, pos, edgelist=path_edges, edge_color='red', width=3, ax=ax2)
        
        # Edge weights
        edge_labels = {}
        for u,v in complete_G.edges():
            weight = next((magnitude for node, magnitude in Graph_nodes[u] if node == v), 0)
            edge_labels[(u,v)] = f'w:{weight}'
        
        nx.draw_networkx_edge_labels(complete_G, pos, edge_labels=edge_labels, ax=ax2)
        
        ax2.set_title('Complete Graph with RBFS Costs')
        ax2.set_xlim(-1.5, 1.5)
        ax2.set_ylim(-1.5, 1.5)
        

        # ax1.text(0.02, 0.98, 'RBFS Legend:\n🟡 Current (Yellow)\n🟢 Open (Green)\n🔴 Closed (Red)\n🔵 Unexplored (Blue)', transform=ax1.transAxes, verticalalignment='top',fontname="Segoe UI Emoji", bbox=dict(boxstyle='round', alpha=0.5, facecolor='white', edgecolor='black'))
        labels = [
            ("Current (Yellow)", "yellow"),
            ("Open (Green)", "lightgreen"),
            ("Closed (Red)", "lightcoral"),
            ("Unexplored (Blue)", "lightblue")
        ]
        
        y = 0.95
        for label, color in labels:
            ax1.text(
            0.05, y, label,
            transform=ax1.transAxes,
            va="center", ha="left", fontsize=12
            )
            ax1.plot(
            0.02, y, "o", color=color, markersize=12,
            transform=ax1.transAxes, clip_on=False
            )
            y -= 0.07


    
    # Create animation
    anim = FuncAnimation(fig, update, frames=len(animation_data['open_sets']), interval=1500, repeat=False, blit=False)
    
    # plt.tight_layout()
    
    # Save animation to prevent garbage collection
    try:
        anim.save('rbfs_animation.gif', writer=PillowWriter(fps=1))
        print("RBFS Animation saved as 'rbfs_animation.gif'")
    except Exception as e:
        print(f"Could not save GIF: {e}")
    
    plt.show()
    
    ###############################################################
    """Simple visualization that shows each step without complex animation"""

    num_steps = min(len(animation_data['open_sets']), 20)
    fig, axes = plt.subplots(2, (num_steps+1)//2, figsize=(18, 12))
    axes = axes.flatten()
    
    complete_G = nx.Graph()
    for node, neighbors in Graph_nodes.items():
        for neighbor, weight in neighbors:
            complete_G.add_edge(node, neighbor, weight=weight)
    
    pos = nx.spring_layout(complete_G, seed=42)
    
    for step in range(num_steps):
        ax = axes[step]
        
        open_set = animation_data['open_sets'][step]
        closed_set = animation_data['closed_sets'][step]
        current_node = animation_data['current_nodes'][step]
        parents = animation_data['parents_maps'][step]
        f_limit_2nd = animation_data['f_limits_alter_costs'][step]
        
        G = nx.DiGraph()
        all_nodes = set(parents.keys())
        for current_node in all_nodes:
            G.add_node(current_node)
        
        for child, parent in parents.items():
            if child != parent:
                weight = 0
                if parent in Graph_nodes:
                    for neighbor, w in Graph_nodes[parent]:
                        if neighbor == child:
                            weight = w
                            break
                G.add_edge(parent, child, weight=weight)
    
        
        # Node colors
        node_colors = []
        for node in G.nodes():
            if node == current_node:
                node_colors.append('yellow')
            elif node in open_set:
                node_colors.append('lightgreen')
            elif node in closed_set:
                node_colors.append('lightcoral')
            else:
                node_colors.append('lightblue')
        
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=600, ax=ax)
        nx.draw_networkx_edges(G, pos, edge_color='gray', arrows=True, arrowsize=15, ax=ax)
        nx.draw_networkx_labels(G, pos, ax=ax)

        # Edge labels
        edge_labels = {(u, v): d['weight'] for u, v, d in G.edges(data=True)}
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, ax=ax)
        
        ax.set_title(f'RBFS Step {step+1}\nCurrent: {current_node}\nF-limit (2nd alter): {f_limit_2nd}')
        ax.set_xlim(-1.5, 1.5)
        ax.set_ylim(-1.5, 1.5)
    
    for i in range(num_steps, len(axes)):
        axes[i].set_visible(False)
    
    # plt.tight_layout()
    plt.show()
    
    return anim

In [2]:
# Run RBFS algorithm
print("Running Recursive Best-First Search (RBFS) Algorithm...")
final_path, animation_data = RBFS_Anim('S', 'G')

if final_path:
    print(f"\nFinal Path: {final_path}")
    print(f"Total Cost: {animation_data['path_costs'][-1].get('G', 'N/A')}")
    
    print("\nCreating RBFS animated and static step-wise visualization...")
    anim = animate_RBFS_search(animation_data, final_path)    
else:
    print("No path found!")

Running Recursive Best-First Search (RBFS) Algorithm...


NameError: name 'RBFS_Anim' is not defined

<br/>

![RBFS_Animated_Algorithm_GIF_Trace](rbfs_animation.gif)

In [6]:
def display_rbfs_tree_animation(animation_data):
    """Text-based RBFS tree visualization"""
    print("\n" + "="*70)
    print("RBFS TEXT-BASED TREE VISUALIZATION")
    print("="*70)
    
    for step in range(len(animation_data['open_sets'])):
        print(f"\n--- RBFS Step {step+1} ---")
        print(f"Current Node: {animation_data['current_nodes'][step]}")
        print(f"Open Set: {animation_data['open_sets'][step]}")
        print(f"Closed Set: {animation_data['closed_sets'][step]}")
        print(f"F-limit (2nd Best Alternative): {animation_data['f_limits_alter_costs'][step]}")
        
        parents = animation_data['parents_maps'][step]
        path_costs = animation_data['path_costs'][step]
        
        print("RBFS Tree Structure:")
        
        # Build tree structure
        tree = {}
        for child, parent in parents.items():
            if parent not in tree:
                tree[parent] = []
            if child != parent:
                tree[parent].append(child)
        
        def print_rbfs_node(current_node, level=0, is_last=True):
            indent = "    " * level
            prefix = "L_______ " if level > 0 else ""
            
            # Node info with RBFS-specific data
            status = ""
            if current_node == animation_data['current_nodes'][step]:
                status = " <---<  (CURRENT)"
            elif current_node in animation_data['open_sets'][step]:
                status = " [OPEN]"
            elif current_node in animation_data['closed_sets'][step]:
                status = " [CLOSED]"
            
            g_val = path_costs.get(current_node, 'Infinity')
            h_val = heuristic(current_node)
            f_val = g_val + h_val if isinstance(g_val, (int, float)) else 'Infinity'
            
            print(f"{indent}{prefix}{current_node} (g={g_val}, h={h_val}, f=g+h={f_val}){status}")
            
            # Print children
            if current_node in tree:
                children = tree[current_node]
                for i, child in enumerate(children):
                    print_rbfs_node(child, level + 1, i == len(children) - 1)
        
        # Find root
        root = next(current_node for current_node, parent in parents.items() if current_node == parent)
        print_rbfs_node(root)
        
        time.sleep(2)
        
# Method 3: Text-based animation for Graph Nodes Traversal step-wise
display_rbfs_tree_animation(animation_data)


RBFS TEXT-BASED TREE VISUALIZATION

--- RBFS Step 1 ---
Current Node: S
Open Set: {'S'}
Closed Set: set()
F-limit (2nd Best Alternative): inf
RBFS Tree Structure:
S (g=0, h=13, f=g+h=13) <---<  (CURRENT)

--- RBFS Step 2 ---
Current Node: S
Open Set: {'S', 'A', 'C'}
Closed Set: set()
F-limit (2nd Best Alternative): inf
RBFS Tree Structure:
S (g=0, h=13, f=g+h=13) <---<  (CURRENT)
    L_______ A (g=1, h=11, f=g+h=12) [OPEN]
    L_______ C (g=5, h=99, f=g+h=104) [OPEN]

--- RBFS Step 3 ---
Current Node: A
Open Set: {'S', 'C'}
Closed Set: {'A'}
F-limit (2nd Best Alternative): 104
RBFS Tree Structure:
S (g=0, h=13, f=g+h=13) [OPEN]
    L_______ A (g=1, h=11, f=g+h=12) <---<  (CURRENT)
    L_______ C (g=5, h=99, f=g+h=104) [OPEN]

--- RBFS Step 4 ---
Current Node: A
Open Set: {'B', 'S', 'C', 'E'}
Closed Set: {'A'}
F-limit (2nd Best Alternative): 104
RBFS Tree Structure:
S (g=0, h=13, f=g+h=13) [OPEN]
    L_______ A (g=1, h=11, f=g+h=12) <---<  (CURRENT)
        L_______ B (g=3, h=6, f=g+h=